# Universidad Nacional de Córdoba 
# Facultad de Matemática, Astronomía, Física y Computación
# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022
# Predicción de incidendes viales en la ciudad de Medellin

Se trabaja con un dataset del portal de datos de la ciudad de Medellin-MEData
[Victimas de Incidentes viales](http://medata.gov.co/dataset/v%C3%ADctimas-en-incidentes-viales)



Mentora: Isabel Mejia

Estudiantes:

Candela

Gustavo

Carina


#Consigna para Informe de Análisis y Visualización
# General
Se propone la elaboración de un informe o presentación en formato estatico  deberá apuntar a un público técnico, pero sin conocimientos específicos en el tema.

No hay límites respecto a la extensión, pero se valorará el poder de síntesis y de comunicación, así como el uso de las herramientas pertinentes para comunicar.


1. Análisis descriptivo.
¿Cuantas son las entradas del dataset y que representa cada una de ellas? realicemos una breve descripción del dataset.

Clasifiquemos las variables de entradas que posee el dataset. Recordar que tenemos variables categoricas, discretas y continuas. Identifiquemos cada una de ellas.¿Cuáles son las variables categoricas? ¿Cuales son categoricas ordinales?



¿Existen valores que no sean validos? ¿Cuáles son? ¿Qué se puede hacer en esos casos?
¿Poseen outliers? ¿Qué técnicas se pueden utilizar para mitigar su impacto?.


Analicemos la frecuencia de incidentes viales por dia,  mes y por año.Revisemos que pares de variables son independientes¿ Podemos apreciar si hay independencia de las variables : dia y mes ,  mes y año? . Adicione los gráficos o tablas que considere oportuno.

Analicemos si existe un relación entre las variables Sexo y Clase_incidente.

¿Cual es la clase de incidente mas frecuente? 

2. Preguntas Disparadoras
¿Como se distribuyen los incidentes viales por comuna y por barrio?

¿Existe una relación entre las variables Clase_incidente y Condicion








In [2]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 8.6 MB/s 
     |████████████████████████████████| 16.7 MB 43.0 MB/s 
     |████████████████████████████████| 6.3 MB 42.5 MB/s 


In [3]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.get_cachedir()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import geopandas as gpd
import fiona

Lectura de datos

In [4]:

invia_df = pd.read_csv('/mede_victimas_inci (1).csv', sep=';')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,11,12,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
invia_df.head()

,Gravedad_victima,Fecha_incidente,Hora_incidente,Clase_incidente,Direccion_incidente,Sexo,Edad,Condicion,Mes,Dia,Num_dia,Hora,Grupo_edad,Año,Radicado,Latitud,Longitud,Comuna,Barrio
0,Heridos,1/01/2014,00:15:00,Otro,CR 49 CL 72,M,17,Motociclista,Ene,Mié,1.0,0,10 - 19,2014.0,1423940,"6,26691466","-75,5590994",04 - Aranjuez,Manrique Central No. 1
1,Heridos,1/01/2014,00:30:00,Atropello,CR 46 CL 98,M,20,Motociclista,Ene,Mié,1.0,0,20 - 29,2014.0,1423921,"6,289353458","-75,55329197",01 - Popular,Moscú No. 2
2,Heridos,1/01/2014,00:30:00,Atropello,CR 46 CL 98,F,18,Peatón,Ene,Mié,1.0,0,10 - 19,2014.0,1423921,"6,289353458","-75,55329197",01 - Popular,Moscú No. 2
3,Heridos,1/01/2014,00:37:00,Atropello,CL 32 CR 84,M,19,Motociclista,Ene,Mié,1.0,0,10 - 19,2014.0,1423849,"6,234327372","-75,60761079",16 - Belén,Las Mercedes
4,Heridos,1/01/2014,00:37:00,Atropello,CL 32 CR 84,M,39,Peatón,Ene,Mié,1.0,0,30 - 39,2014.0,1423849,"6,234327372","-75,60761079",16 - Belén,Las Mercedes


In [5]:
# Obtenemos la cantidad de filas y columnas del Dataset Original
_shape = invia_df.shape
print("Nuestro Dataset original tiene", _shape[0], "filas y", _shape[1],"columnas.")

Nuestro Dataset original tiene 1048317 filas y 19 columnas.


Veamos el dataset de incidentes viales,   el tipo de datos que tiene! 

In [19]:
data_types = invia_df.dtypes
data_types = pd.DataFrame(data_types)
data_types = data_types.rename(columns = {0:'ReadingDataType'})
data_types.index.names = ['FieldName']
data_types

,ReadingDataType
FieldName,
Gravedad_victima,object
Fecha_incidente,object
Hora_incidente,object
Clase_incidente,object
Direccion_incidente,object
Sexo,object
Edad,object
Condicion,object
Mes,object


Podemos ver que existen muchas de las variables que no se corresponden al tipo de dato especificado:

Fecha_incidente	
Hora_incidente

Edad

Latitud

Longitud



In [20]:
# Convertimos de object a datetime

invia_df['Fecha_incidente'] = pd.to_datetime(invia_df.Fecha_incidente, dayfirst=True, format='%d/%m/%Y')



In [22]:
data_types = invia_df.dtypes
data_types = pd.DataFrame(data_types)
data_types = data_types.rename(columns = {0:'ReadingDataType'})
data_types.index.names = ['FieldName']
data_types

,ReadingDataType
FieldName,
Gravedad_victima,object
Fecha_incidente,datetime64[ns]
Hora_incidente,object
Clase_incidente,object
Direccion_incidente,object
Sexo,object
Edad,object
Condicion,object
Mes,object


#La prueba de chi-cuadrado
 es un método de prueba de hipótesis ampliamente utilizado para contar datos. Pertenece a la categoría de pruebas no paramétricas y compara principalmente dos o más frecuencias de muestreo (proporciones de composición) y el análisis de correlación de dos variables categóricas. La idea básica es comparar el grado de ajuste o la bondad del ajuste entre la frecuencia teórica y la frecuencia real.

 Les comparto el siguiente link donde se aplica la prueba [Chi-cuadrado](https://isolution.pro/es/t/python-data-science/python-chi-square-test/python-prueba-de-chi-cuadrado#:~:text=Python%20-%20Prueba%20de%20chi-cuadrado%20La%20prueba%20de,variables%20categ%C3%B3ricas%20tienen%20una%20correlaci%C3%B3n%20significativa%20entre%20ellas.) 